In [1]:
from onnx import TensorProto
from qonnx.core.modelwrapper import ModelWrapper
from slice_template import slice_node
model = ModelWrapper("tinyyolo-20210831_v081.onnx")
# model.save("tinyyolo-20210831_v081.onnx")

In [3]:
from onnx import TensorProto
from qonnx.core.modelwrapper import ModelWrapper
from slice_template import slice_node

# Open the Tinyyolo model
tmp_model = ModelWrapper("tinyyolo-20210831.onnx")

# Create a dictionary to store the slice nodes in 
update_dict = {}
# These are the inputs of a slice node in the correct order
key_list = ["input", "starts", "ends", "axes", "splits", "output"]
# These are the tensors from the replaced slice nodes
drop_tensors = []

# Loop over all the nodes in the graph at are of op_type Slice
for s in tmp_model.get_nodes_by_op_type("Slice"):
    # Get the node index (maybe can remove?!)
    node_inx = tmp_model.get_node_index(s)
    # Store all tensor value info, initialization, and shapes for the current slice node
    vinfo = [tmp_model.get_tensor_valueinfo(inp) for inp in s.input]
    init = [tmp_model.get_initializer(inp) for inp in s.input]
    shapes = [tmp_model.get_tensor_shape(i) for i in s.input]
    # Get the input tensor names
    t_names = [i for i in s.input]
    
    # Create a dict of attributes and initalize an empty dict [Tensor_name, shape, initialization value]
    attr = {}
    for k in key_list:
        attr[k] = [None, None, None]
    
    # Loop over the len of shapes (not all slice nodes contain a splits value)
    for ind in range(len(shapes)):
        # Store the vlaue in attribute
        attr[key_list[ind]] = [t_names[ind], shapes[ind], init[ind]]
        # Append list of tensors for this node
        drop_tensors.append(t_names[ind])

    # Do the same as above but for the output of the slice node
    for i in s.output:
        attr["output"] = [i, tmp_model.get_tensor_shape(i)]
        drop_tensors.append(i)
    # Print some stats
    print("*"*110)
    print(attr["starts"][2][0])
    print(attr)
    # Create a new slice node with attr instead of inputs
    x = slice_node(input_shape=attr["input"][1], output_shape=attr["output"][1],
                   starts_value=attr["starts"][2], ends_value=attr["ends"][2], axes_value=attr["axes"][2], 
                   splits_value=attr["splits"][2],input_tensor=attr["input"][0], output_tensor=attr["output"][0],
                   dtype=TensorProto.INT64,node_name = s.name)
    # Update dict is a dict that stores the new slice node with key = slice name, index and node
    update_dict[s.name] = [node_inx, x.make_node()]
    # store the model of the single slice node. This is for double checking
    x.make_model(s.name+".onnx")


**************************************************************************************************************
0
{'input': ['673', None, None], 'starts': ['675', [1], array([0])], 'ends': ['676', [1], array([2])], 'axes': ['674', [1], array([4])], 'splits': ['677', [1], array([1])], 'output': ['678', None]}
[1]


TypeError: iteration over a 0-d array

# Create a new model
In this part of the notebook I will create a new model by removing the old slice nodes from the onnx file and replacing them with the new slice nodes created earlier in this notebook

In [10]:
import onnx
from qonnx.transformation.infer_shapes import InferShapes
# Load in the tinyyolo model
model = ModelWrapper("tinyyolo-20210831_v081.onnx")

# Create empty lists 
vinfo, keep_nodes, new_nodes = [], [], []
# Create an empty dict for initialization tensors
t_init = {}
tensors = model.get_all_tensor_names()

glb_in = model.graph.input[0]
glb_out = model.graph.output[0]
drop_tensors.append(glb_in.name)
drop_tensors.append(glb_out.name)
# Loop over all tenors in the model and remove the ones that belong to the dropped nodes
for t in tensors:
    # Store the value info and initialization values if they exist.
    if not (t in drop_tensors):
        vinfo.append(model.get_tensor_valueinfo(t))
        if not model.get_initializer(t) is None:
            t_init[t] = model.get_initializer(t)
# These are the nodes that need to be removed
replace_nodes = update_dict.keys()

# Make a list of nodes to keep
for n in model.graph.node:
    if not n.name in replace_nodes:
        keep_nodes.append(n)

# Make a list of the new nodes
new_nodes = [v[1] for k,v in update_dict.items()]
for nnode in new_nodes:
    keep_nodes.append(nnode)
    
model_config = {}
model_config["opset_imports"] = [onnx.helper.make_operatorsetid("",11)]
# Create a new model
graph = onnx.helper.make_graph(
        nodes= keep_nodes,
        name="tinyyolo_infershapes_updated",
        inputs=[model.graph.input[0]],
        outputs=[model.graph.output[0]],
        value_info=vinfo[0:-2]
        )
new_model = ModelWrapper(onnx.helper.make_model(graph,**model_config))
new_model.model.ir_version = 6
print(new_model.model.ir_version)

for k,v in t_init.items():
    new_model.set_initializer(k,v)

for t in tensors:
    dt = model.get_tensor_datatype(t)
    if not dt == "FLOAT32":
        new_model.set_tensor_datatype(t,dt)
        
new_model.save("tinyyolo_slice_update.onnx")
infer_shapes = new_model.transform(InferShapes())


6


In [ ]:
# %%time
! python build.py
# version = {}
# version["ir_version"] = "1.6.0"
# onnx.helper.make_model_gen_version(graph,**version)

Building dataflow accelerator from tinyyolo_slice_update.onnx
Intermediate outputs will be generated in /workspace/results
Final outputs will be generated in build-tinyyolo_slice_update
Build log is at build-tinyyolo_slice_update/build_dataflow.log
Running step: step_qonnx_to_finn [1/17]
Running step: step_tidy_up [2/17]
Traceback (most recent call last):
  File "/workspace/finn/src/finn/builder/build_dataflow.py", line 166, in build_dataflow_cfg
    model = transform_step(model, cfg)
  File "/workspace/finn/src/finn/builder/build_dataflow_steps.py", line 243, in step_tidy_up
    model = model.transform(FoldConstants())
  File "/workspace/qonnx/src/qonnx/core/modelwrapper.py", line 140, in transform
    (transformed_model, model_was_changed) = transformation.apply(transformed_model)
  File "/workspace/qonnx/src/qonnx/transformation/fold_constants.py", line 59, in apply
    oxe.execute_node(n, execution_context, graph)
  File "/workspace/qonnx/src/qonnx/core/onnx_exec.py", line 74, in e